### NOTEBOOK for constructiong DMD NIROM approximation for a shallow water example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. DMD is adopted to both construct a latent space representation of the high-fidelity temporal snapshots and to approximate the evolution of the time dynamics.  




In [ ]:
## Load all modules
%matplotlib inline
import numpy as np
import scipy
from scipy import linalg
import matplotlib.pyplot as plt

import scipy.interpolate as interpolate
import bisect
import gc
import os,sys
from importlib import reload

# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, # fontsize for x and y labels (was 10)
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

basedir = os.getcwd()
workdir = os.path.join(basedir,'../examples/')
datadir = os.path.join(basedir,'../data/')
nirom_data_dir = os.path.join(basedir,'../data/')
figdir = os.path.join(basedir,'../figures/dmd/')


import pynirom
from pynirom.dmd import main as dmd
from pynirom.dmd import plotting as pu
from pynirom.utils import data_utils as du


In [ ]:
## Load snapshot data
## Please download the data files from 
## https://drive.google.com/drive/folders/1yhudg8RPvwV9SJx9CTqANEnyN55Grzem?usp=sharing
## and place them in <data_dir>

# ### San Diego problem
model = 'SD'
data = np.load(os.path.join(datadir,'san_diego_tide_snapshots_T4.32e5_nn6311_dt25.npz'))
mesh = np.load(os.path.join(datadir,'san_diego_mesh.npz'))

# ### Red River problem
# model ='RED'
# data = np.load(os.path.join(datadir,'red_river_inset_snapshots_T7.0e4_nn12291_dt10.npz'))
# mesh = np.load(os.path.join(datadir,'red_river_mesh.npz'))

print("Solution component keys are : " + str(list(data.keys())))
print("Mesh element keys are : " + str(list(mesh.keys())))

print('\nHFM data has {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    data['T'].shape[0],data['S_dep'].shape[0],
                    data['T'][0], data['T'][-1]))



In [ ]:
## Prepare training and testing snapshots
print('\n-------Prepare training and testing data---------')
soln_names = ['S_dep', 'S_vx', 'S_vy']
comp_names={0:'S_dep',1:'S_vx',2:'S_vy'}
Nc=3 
nodes = mesh['nodes']; triangles = mesh['triangles']
Nn = nodes.shape[0]; Ne = triangles.shape[0]
snap_start = 100

## ------- Save full HFM data without spinup time -----
snap_data, times_offline = du.prepare_data(data, soln_names, start_skip=snap_start,)


## ----- Prepare training snapshots
if model == 'SD':
    T_end = 50*3600   ### 50 hours in seconds
    snap_incr=4       ### number of steps to skip in selecting training snapshots for SVD basis
elif model == 'RED':
    T_end = 3.24e4    ### 9 hours in seconds
    snap_incr=3       ### number of steps to skip in selecting training snapshots for SVD basis

snap_train, times_train = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_train.shape[0],
                                        times_train[0]/3600, times_train[-1]/3600))


DT = (times_offline[1:] - times_offline[:-1]).mean()
Nt = times_offline.size
## ---- Normalize the time axis. Required for DMD fitting
tscale = DT*snap_incr                  ### Scaling for DMD
times_offline = times_offline/tscale   ## Snapshots DT = 1
times_train=times_train/tscale
Nt_b = times_train.size


## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr - 2
snap_pred_true, times_online = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, 
                                                    incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_online.shape[0],
                                        times_online[0]/3600, times_online[-1]/3600))
times_online = times_online/tscale
Nt_online=times_online.size



del data
del mesh
gc.collect()

In [ ]:
### Set up the snapshot data matrices X and Y describing the DMD flow map : Y = AX
## For multicomponent systems such Navier Stokes equations (p,u,v),
## the snapshot matrix can be created either by vertically stacking
## solution variables one after another, or by interleaving the
## solution variables at every spatial node.
## In each case, the snapshot matrix has dimensions
## (3 * DOF) X (Time Steps)

interleaved_snapshots = True
X0 = np.zeros((Nc*Nn,Nt_b),'d')

for ivar,key in enumerate(soln_names):   
    if interleaved_snapshots:    ### saving in an interleaved fashion
        X0[ivar::Nc,:] = snap_train[key][:,:]
    else:                        ### saving in a sequential fashion
        X0[ivar*Nn:(ivar+1)*Nn,:] = snap_train[key][:,:]

        
X  = X0[:,:-1]
Xp = X0[:,1:]

In [ ]:
## Create a snapshot matrix for the true solution at 
## testing time points, following the chosen 
## stacking convention

X_true = np.zeros((Nc*Nn,Nt_online),'d')

for ivar,key in enumerate(soln_names):
    ### saving in an interleaved fashion
    if interleaved_snapshots:    
        X_true[ivar::Nc,:] = snap_pred_true[key] 
    ### saving in a sequential fashion
    else:                        
        X_true[ivar*Nn:(ivar+1)*Nn,:] = snap_pred_true[key]


In [ ]:
def var_string(ky):
    if ky == 'S_dep':
        md = 'h'
    elif ky == 'S_vx':
        md = 'u'
    elif ky == 'S_vy':
        md = 'v'
    return md

In [ ]:
### Compute the DMD modes
## Using a predetermined fixed number of truncation modes for SVD

if model == 'RED':
    r = 315 #30
elif model == 'SD':
    r = 115 #250
t0,dt = times_train[0], times_train[1] - times_train[0]

DMD=dmd.DMDBase(rank=r)
Phi,D,X_app,td,pod_Sigma = DMD.fit_basis(X0, dt_fit = dt, t0_fit=times_train[0])

Xdmd = np.zeros((Nn*Nc,Nt_online),'d')
for inx,tn in enumerate(times_online):
    Xdmd[:,inx] = DMD.predict(tn)
print("DMD snapshots computed for %d steps between t = [%.3f, %.3f]"%(Nt_online, 
                                    times_online[0]*tscale, times_online[-1]*tscale))


In [ ]:
### Look at the singular value decay
pu.plot_sing_val(pod_Sigma[:r],6,4)

In [ ]:
## Visualize distribution of DMD e-values
pu.plot_DMD_eval(D,r)

print('Total distance between eigenvalues and unit circle: {}'.format(
    np.sum(np.abs(D.real**2 + D.imag**2 - 1)) ) )


In [ ]:
def plot_dmd_soln(X, Xdmd, Nc, Nt_plot, nodes, elems, times_online, comp_names, seed, flag = True): 
    
    np.random.seed(seed)
    itime = 1200 #np.random.randint(0,Nt_plot)
    ivar  = 1 #np.random.randint(1,Nc)

    if flag:     ### for interleaved snapshots
        tmp      = Xdmd[ivar::Nc,itime]
        tmp_snap = X[ivar::Nc,itime]
    else:
        tmp      = Xdmd[ivar*Nn:(ivar+1)*Nn,itime]
        tmp_snap = X[ivar*Nn:(ivar+1)*Nn,itime]

    ky = comp_names[ivar]
    tn   = times_online[itime]*tscale
    
    fig  = plt.figure(figsize=(18,15));
    ax1   = fig.add_subplot(2, 2, 1)
    surf1 = ax1.tripcolor(nodes[:,0], nodes[:,1],elems, tmp, cmap=plt.cm.jet)
    ax1.set_title('DMD solution: {0} at t={1:1.2f} hrs,\n {0} range = [{2:5.3g},{3:4.2g}]'.format(var_string(ky),
                                                                        tn/3600,
                                                                        tmp.min(),tmp.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf1, shrink=0.8,aspect=20, pad = 0.03)

    ax2   = fig.add_subplot(2, 2, 2)
    surf2 = ax2.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_snap, cmap=plt.cm.jet)
    ax2.set_title('HFM solution: {0} at t={1:1.2f} hrs,\n {0} range = [{2:5.3g},{3:4.2g}]'.format(var_string(ky),
                                                                    tn/3600,
                                                                    tmp_snap.min(),tmp_snap.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf2, shrink=0.8,aspect=20, pad = 0.03)

    err = tmp-tmp_snap
    ax3   = fig.add_subplot(2, 2, 3)
    surf3 = ax3.tripcolor(nodes[:,0], nodes[:,1],elems, err, cmap=plt.cm.Spectral)
    ax3.set_title('DMD error: {0} at t={1:1.2f} hrs,\n error range = [{2:5.3g},{3:4.2g}]'.format(var_string(ky),
                                                                    tn/3600,
                                                                    err.min(),err.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf3,shrink=0.8,aspect=20, pad = 0.03)
    
#     fig.tight_layout()
    return tn, ky


In [ ]:
Nt_plot = np.searchsorted(times_online, times_train[-1]); 
tn,ky = plot_dmd_soln(X_true, Xdmd, Nc, Nt_plot, nodes, triangles, times_online, comp_names, seed=1234,flag = True)

# os.chdir(figdir)
# plt.savefig('%s_dmd_%s_t%.2f_tskip%d_oskip%d_r%d.pdf'%(model,ky,tn,snap_incr,pred_incr,r), bbox_inches='tight')


In [ ]:
def plot_vel_mag(X, Xdmd, Nc, Nt_plot, nodes, elems, times_online, flag = True):
    '''
    Plot the magnitude of the velocity for the true solution, 
    the DMD solution and the error
    '''
    import math
    from math import hypot
    
    np.random.seed(1234)
    itime = 1200 #np.random.randint(0,Nt_plot)

    if flag:   ## snapshots are stored in an interleaved fashion
        tmp      = np.sqrt(Xdmd[0::Nc,itime]**2 + Xdmd[1::Nc,itime]**2)
        tmp_snap = np.sqrt(X[0::Nc,itime]**2 + X[1::Nc,itime]**2)
    else:
        tmp      = Xdmd[ivar*Nn:(ivar+1)*Nn,itime]
        tmp_snap = X[ivar*Nn:(ivar+1)*Nn,itime]

    tn   = times_online[itime]*tscale
    fig  = plt.figure(figsize=(18,15));
    ax   = fig.add_subplot(2, 2, 1)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, tmp, cmap=plt.cm.jet)
    ax.set_title('DMD solution: $|u|$ at t={0:1.2f} hrs,\n $|u|$ range = [{1:5.3g},{2:4.2g}]'.format(tn/3600,
                                                                    tmp.min(),tmp.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf, shrink=0.8,aspect=20, pad = 0.03)


    ax   = fig.add_subplot(2, 2, 2)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_snap, cmap=plt.cm.jet)
    ax.set_title('HFM solution: $|u|$ at t={0:1.2f} hrs,\n $|u|$ range = [{1:5.3g},{2:4.2g}]'.format(tn/3600,
                                                                    tmp_snap.min(),tmp_snap.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf, shrink=0.8,aspect=20, pad = 0.03)

    
    err = tmp-tmp_snap
    rel_err = err/tmp_snap
    ax   = fig.add_subplot(2, 2, 3)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, err, cmap=plt.cm.Spectral)
    ax.set_title('DMD rel. error: $|u|$ at t={0:1.2f} hrs,\n rel. err. range = [{1:5.3g},{2:4.2g}]'.format(tn/3600,
                                                                    err.min(),err.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf,shrink=0.8,aspect=20, pad = 0.03)



In [ ]:
Nt_plot = np.searchsorted(times_online, times_train[-1])
plot_vel_mag(X_true, Xdmd, Nc, Nt_online, nodes, triangles, times_online)

In [ ]:
### Visualize spatial RMS/Relative errors
x_inx = times_online*tscale/3600
tr_mark = np.searchsorted(times_online, times_train[-1])

dmd_rms = DMD.compute_error(X_true, Xdmd, soln_names, metric = 'rms')

vstring = {}
for key in soln_names:
    vstring[key] = var_string(key)
    
pu.plot_DMD_err(dmd_rms, x_inx, soln_names, vstring, unit='hours' )

# os.chdir(figdir)
# plt.savefig('%s_dmd_rms_tskip%d_oskip%d_r%d.pdf'%(model,snap_incr,pred_incr,r),bbox_extra_artists=(lg,), bbox_inches='tight')

In [ ]:
# Save the NIROM solutions to disk
save_nirom_solutions = False

if save_nirom_solutions:
    os.chdir(nirom_data_dir)
    np.savez_compressed('%s_online_dmd_r%d'%(model,r),dmd=Xdmd, true=X_true,time=times_online,tscale=tscale,r=r,
                        interleaved=interleaved_snapshots)

    os.chdir(workdir)

In [ ]:
## Saving the ROM model
# os.chdir(nirom_data_dir)
# if model == 'SD':
#     filename='dmd_rom_sandiego'
# elif model == 'RED':
#     filename='dmd_rom_redriver'
# DMD.save_to_disk(filename,DMD)